# 🎯 Image Classification Demo

Use classic image features with scikit-learn for image classification.

We'll use a subset of **CIFAR-10** to demonstrate feature extraction and classification.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kelkalot/imagefeatures/blob/main/examples/demo_classification.ipynb)

## 1. Setup

In [ ]:
!pip install -q git+https://github.com/kelkalot/imagefeatures.git
!pip install -q scikit-learn matplotlib

## 2. Load CIFAR-10 Subset

We'll use a small subset (500 images) for quick demonstration.

In [ ]:
import numpy as np
import urllib.request
import pickle
import tarfile
import os

# Download CIFAR-10 if needed
cifar_url = 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'
cifar_file = 'cifar-10-python.tar.gz'

if not os.path.exists('cifar-10-batches-py'):
    print('Downloading CIFAR-10...')
    urllib.request.urlretrieve(cifar_url, cifar_file)
    with tarfile.open(cifar_file, 'r:gz') as tar:
        tar.extractall()
    os.remove(cifar_file)
    print('Done!')

# Load one batch
def load_cifar_batch(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
    return data[b'data'], data[b'labels']

X_raw, y = load_cifar_batch('cifar-10-batches-py/data_batch_1')

# Reshape to images (N, 32, 32, 3)
X_images = X_raw.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)

# Use subset for speed
n_samples = 500
X_images = X_images[:n_samples]
y = np.array(y[:n_samples])

# Class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
               'dog', 'frog', 'horse', 'ship', 'truck']

print(f'Loaded {len(X_images)} images')
print(f'Image shape: {X_images[0].shape}')
print(f'Classes: {len(set(y))} ({len(class_names)} total)')

## 3. Display Sample Images

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    idx = np.where(y == i)[0][0]  # First image of each class
    ax.imshow(X_images[idx])
    ax.set_title(class_names[i])
    ax.axis('off')
plt.suptitle('CIFAR-10 Sample (one per class)', fontsize=14)
plt.tight_layout()
plt.show()

## 4. Extract Features

We'll extract a compact set of features suitable for small images.

In [ ]:
from imagefeatures.features import (
    ColorHistogram, ColorMoments, LocalBinaryPatterns,
    Gabor, Tamura, CEDD, Haralick, HuMoments
)

# Define feature extractors (compact features for 32x32 images)
extractors = [
    ColorHistogram(),    # 64d - color distribution
    ColorMoments(),      # 9d  - color statistics
    LocalBinaryPatterns(), # 256d - texture
    Gabor(),             # 48d - multi-scale texture
    Haralick(),          # 6d  - GLCM texture
    HuMoments(),         # 7d  - shape
]

total_dim = sum(e.dim for e in extractors)
print(f'Feature extractors: {len(extractors)}')
print(f'Total dimensions: {total_dim}')
for e in extractors:
    print(f'  - {e.name}: {e.dim}d')

In [ ]:
# Extract features from all images
print(f'Extracting features from {len(X_images)} images...')

X_features = []
for i, img in enumerate(X_images):
    if i % 100 == 0:
        print(f'  Processing image {i}/{len(X_images)}')
    
    features = []
    for extractor in extractors:
        extractor.extract(img)
        features.append(extractor.get_feature_vector())
    
    X_features.append(np.concatenate(features))

X_features = np.array(X_features)
print(f'\nFeature matrix shape: {X_features.shape}')

## 5. Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_features, y, test_size=0.2, random_state=42, stratify=y
)

# Normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f'Training set: {len(X_train)} images')
print(f'Test set: {len(X_test)} images')

## 6. Train Classifiers

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

classifiers = {
    'k-NN (k=5)': KNeighborsClassifier(n_neighbors=5),
    'SVM (RBF)': SVC(kernel='rbf', C=10, gamma='scale'),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
}

results = {}

for name, clf in classifiers.items():
    print(f'Training {name}...')
    clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    print(f'  Accuracy: {acc:.2%}\n')

## 7. Compare Results

In [ ]:
import matplotlib.pyplot as plt

# Bar chart of accuracies
fig, ax = plt.subplots(figsize=(10, 5))
names = list(results.keys())
accs = list(results.values())
colors = plt.cm.viridis(np.linspace(0.3, 0.8, len(names)))

bars = ax.barh(names, accs, color=colors)
ax.set_xlim(0, 1)
ax.set_xlabel('Accuracy')
ax.set_title('CIFAR-10 Classification Results (500 samples)')

# Add value labels
for bar, acc in zip(bars, accs):
    ax.text(acc + 0.02, bar.get_y() + bar.get_height()/2, 
            f'{acc:.1%}', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

print(f'\nBest classifier: {max(results, key=results.get)} ({max(results.values()):.1%})')

## 8. Detailed Classification Report

In [ ]:
# Use best classifier for detailed report
best_clf_name = max(results, key=results.get)
best_clf = classifiers[best_clf_name]
y_pred = best_clf.predict(X_test_scaled)

print(f'Classification Report ({best_clf_name}):\n')
print(classification_report(y_test, y_pred, target_names=class_names))

## 9. Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred)

fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(cm, cmap='Blues')

# Labels
ax.set_xticks(range(10))
ax.set_yticks(range(10))
ax.set_xticklabels(class_names, rotation=45, ha='right')
ax.set_yticklabels(class_names)
ax.set_xlabel('Predicted')
ax.set_ylabel('True')
ax.set_title(f'Confusion Matrix ({best_clf_name})')

# Add values
for i in range(10):
    for j in range(10):
        color = 'white' if cm[i, j] > cm.max()/2 else 'black'
        ax.text(j, i, cm[i, j], ha='center', va='center', color=color)

plt.colorbar(im, fraction=0.046)
plt.tight_layout()
plt.show()

## 10. Feature Importance (Random Forest)

In [ ]:
# Get feature importances from Random Forest
rf = classifiers['Random Forest']
importances = rf.feature_importances_

# Group by feature extractor
feature_groups = {}
idx = 0
for extractor in extractors:
    dim = extractor.dim
    group_importance = np.sum(importances[idx:idx+dim])
    feature_groups[extractor.name] = group_importance
    idx += dim

# Plot
fig, ax = plt.subplots(figsize=(10, 5))
names = list(feature_groups.keys())
values = list(feature_groups.values())
colors = plt.cm.Spectral(np.linspace(0.1, 0.9, len(names)))

bars = ax.barh(names, values, color=colors)
ax.set_xlabel('Total Feature Importance')
ax.set_title('Feature Group Importance (Random Forest)')

for bar, val in zip(bars, values):
    ax.text(val + 0.01, bar.get_y() + bar.get_height()/2, 
            f'{val:.2f}', va='center')

plt.tight_layout()
plt.show()

## 11. Key Takeaways

- **Classic features work!** Even with 32x32 images, we can achieve reasonable accuracy.
- **Feature combination matters**: Mixing color, texture, and shape features improves results.
- **SVM often works best** with classic features due to the normalized feature space.
- **Random Forest** provides valuable feature importance insights.

### Tips for Better Results:
1. **More features**: Add CEDD, FCTH, PHOG for richer representation
2. **Larger dataset**: Use full CIFAR-10 (50,000 images)
3. **Feature selection**: Remove low-importance features
4. **Hyperparameter tuning**: Use GridSearchCV for optimal parameters